# Spotify API

In this notebook, I use the spotify API to scrape data from multiple spotify playlists. By doing this, I will create a dataframe of more than thousands of songs.

In [1]:
import pandas as pd
import numpy as np

## Import sp and authenticate with the secret

In [188]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials #this line will enable me to authenticate myself


In [189]:
secrets_file = open("secrets.txt","r")

In [ ]:
string = secrets_file.read()

string

In [ ]:
string.split('\n')


In [ ]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip() # strip is if there was a space in the passcode.

secrets_dict

## Authentication

In [193]:
import spotipy #wrapper
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['csecret']))

## Exploring (and the dataframe will be created in the next section)

### Playlists

In [45]:
playlist = sp.user_playlist_tracks("spotify", "36f5MuoelMRwVIet1HUqYC") #default limit in playlists is 100


In [ ]:
playlist

In [47]:
playlist.keys() 

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [48]:
playlist["items"][0].keys()

dict_keys(['added_at', 'added_by', 'is_local', 'primary_color', 'track', 'video_thumbnail'])

In [ ]:
playlist["items"][0]["track"]

In [50]:
playlist["items"][0]["track"]["name"]

'Imagine - Remastered 2010'

In [51]:
playlist["total"] 

502

In [52]:
len(playlist["items"])

100

In [53]:
from random import randint
from time import sleep

def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3)) # 1, 2, or 3 seconds
    return tracks

In [54]:
all_tracks = get_playlist_tracks("36f5MuoelMRwVIet1HUqYC")
len(all_tracks)

502

In [ ]:
all_tracks

In [56]:
all_tracks[501]['track']['album']['artists'][0]['name'] # artist names

'Nick Drake'

In [57]:
all_tracks[10]['track']['name']#.keys()#['album']['name'] # song names

'The Show Must Go On - Remastered 2011'

In [ ]:
all_tracks # is a list of dictionaries!

### Audio Features

In [59]:
all_tracks[0].keys()

dict_keys(['added_at', 'added_by', 'is_local', 'primary_color', 'track', 'video_thumbnail'])

In [60]:
all_tracks[501]['track']['uri']

'spotify:track:6JFlwCkZGXj62ObbD9zDeX'

In [61]:
song_uri= all_tracks[500]['track']['uri']

In [62]:
features=sp.audio_features(song_uri)

In [63]:
features[0]

{'danceability': 0.186,
 'energy': 0.158,
 'key': 2,
 'loudness': -18.775,
 'mode': 0,
 'speechiness': 0.0316,
 'acousticness': 0.84,
 'instrumentalness': 0.595,
 'liveness': 0.0708,
 'valence': 0.0652,
 'tempo': 86.651,
 'type': 'audio_features',
 'id': '6p2AmqmBpr2EPgohLqIS7A',
 'uri': 'spotify:track:6p2AmqmBpr2EPgohLqIS7A',
 'track_href': 'https://api.spotify.com/v1/tracks/6p2AmqmBpr2EPgohLqIS7A',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6p2AmqmBpr2EPgohLqIS7A',
 'duration_ms': 618640,
 'time_signature': 3}

In [64]:
playlist['items'][0]['track']['uri']

'spotify:track:7pKfPomDEeI4TPT6EOYjn9'

## Creating a df with artist song and audio features columns

### First playlist

In [65]:
from random import randint
from time import sleep

def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3)) # 1, 2, or 3 seconds
    return tracks

In [66]:
all_tracks = get_playlist_tracks("36f5MuoelMRwVIet1HUqYC")

In [67]:
len(all_tracks)

502

In [68]:
def flatten_it(data, col_list):
    for column in col_list:
        flattened = pd.DataFrame(dict(data[column])).transpose()
        columns = [str(col) for col in flattened.columns]
        flattened.columns = [column + '_' + colname for colname in columns]
        data = pd.concat([data, flattened], axis=1)
        data = data.drop(column, axis=1)
    return data

In [69]:
def playlist_to_df(tracks_list):
    artist_s = []
    song_s = []
    song_uri_s=[]
    features_s= []

    num_iter = len(tracks_list)

    for i in range(num_iter):
        artist_s.append(tracks_list[i]['track']['album']['artists'][0]['name'])
        song_s.append(tracks_list[i]['track']['name'])
        song_uri_s.append(tracks_list[i]['track']['uri'])
        features_s.append(sp.audio_features(song_uri_s[i]))
        
    df_v0= pd.DataFrame({"artist":artist_s,"song":song_s,"audio_features":features_s})
    
    nested_columns = ['audio_features']
    df_v1 = flatten_it(df_v0, nested_columns)
    music_df=pd.concat([df_v1.drop(['audio_features_0'], axis=1), df_v1['audio_features_0'].apply(pd.Series)], axis=1)
    
    return music_df


In [70]:
spotify_df_1=playlist_to_df(all_tracks)

In [71]:
display(spotify_df_1.head())
display(spotify_df_1.shape)


,artist,song,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,John Lennon,Imagine - Remastered 2010,0.547,0.257,0,-12.358,1,0.0252,0.907,0.1830,0.0935,0.169,75.752,audio_features,7pKfPomDEeI4TPT6EOYjn9,spotify:track:7pKfPomDEeI4TPT6EOYjn9,https://api.spotify.com/v1/tracks/7pKfPomDEeI4...,https://api.spotify.com/v1/audio-analysis/7pKf...,187867,4
1,Procol Harum,A Whiter Shade Of Pale,0.249,0.660,0,-6.905,1,0.0342,0.504,0.0026,0.0891,0.435,149.813,audio_features,3qitymULqEibr7yknRMKU4,spotify:track:3qitymULqEibr7yknRMKU4,https://api.spotify.com/v1/tracks/3qitymULqEib...,https://api.spotify.com/v1/audio-analysis/3qit...,248947,4
2,Queen,Bohemian Rhapsody - Remastered 2011,0.414,0.404,0,-9.928,0,0.0499,0.271,0.0000,0.3000,0.224,71.105,audio_features,1AhDOtG9vPSOmsWgNW0BEY,spotify:track:1AhDOtG9vPSOmsWgNW0BEY,https://api.spotify.com/v1/tracks/1AhDOtG9vPSO...,https://api.spotify.com/v1/audio-analysis/1AhD...,354320,4
3,Pink Floyd,Comfortably Numb - 2011 Remastered Version,0.472,0.366,11,-12.595,0,0.0286,0.150,0.3080,0.0837,0.171,127.167,audio_features,082cLCIXNPg2ruTrENz4Vt,spotify:track:082cLCIXNPg2ruTrENz4Vt,https://api.spotify.com/v1/tracks/082cLCIXNPg2...,https://api.spotify.com/v1/audio-analysis/082c...,382297,4
4,Bob Dylan,Knockin' On Heaven's Door,0.513,0.396,7,-13.061,1,0.0299,0.251,0.1770,0.1100,0.229,140.208,audio_features,6HSXNV0b4M4cLJ7ljgVVeh,spotify:track:6HSXNV0b4M4cLJ7ljgVVeh,https://api.spotify.com/v1/tracks/6HSXNV0b4M4c...,https://api.spotify.com/v1/audio-analysis/6HSX...,149880,4


(502, 20)

### Adding a second playlist

In [72]:
all_tracks_2 = get_playlist_tracks("532F1h299qKD894BlPfJJF")

In [73]:
len(all_tracks_2)

6246

In [74]:
spotify_df_2=playlist_to_df(all_tracks_2)

In [75]:
display(spotify_df_2.head())
display(spotify_df_2.shape)


,artist,song,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Roy Orbison,"Oh, Pretty Woman",0.619,0.603,9,-9.481,1,0.0342,0.712,0.000000,0.0721,0.958,127.433,audio_features,48i055G1OT5KxGGftwFxWy,spotify:track:48i055G1OT5KxGGftwFxWy,https://api.spotify.com/v1/tracks/48i055G1OT5K...,https://api.spotify.com/v1/audio-analysis/48i0...,178933,4
1,The Monkees,I'm a Believer - 2006 Remaster,0.526,0.775,0,-6.050,1,0.0352,0.707,0.000021,0.2170,0.962,80.106,audio_features,3G7tRC24Uh09Hmp1KZ7LQ2,spotify:track:3G7tRC24Uh09Hmp1KZ7LQ2,https://api.spotify.com/v1/tracks/3G7tRC24Uh09...,https://api.spotify.com/v1/audio-analysis/3G7t...,167373,4
2,Jefferson Airplane,Somebody to Love,0.504,0.578,4,-9.302,1,0.0310,0.408,0.000000,0.3210,0.654,132.411,audio_features,4uGIJG1jYFonGc4LGp5uQL,spotify:track:4uGIJG1jYFonGc4LGp5uQL,https://api.spotify.com/v1/tracks/4uGIJG1jYFon...,https://api.spotify.com/v1/audio-analysis/4uGI...,174840,4
3,The Turtles,Happy Together,0.584,0.367,6,-9.638,0,0.0328,0.550,0.000014,0.0818,0.588,120.175,audio_features,1JO1xLtVc8mWhIoE3YaCL0,spotify:track:1JO1xLtVc8mWhIoE3YaCL0,https://api.spotify.com/v1/tracks/1JO1xLtVc8mW...,https://api.spotify.com/v1/audio-analysis/1JO1...,176293,4
4,The Beach Boys,Wouldn't It Be Nice - Remastered 2000 / Stereo...,0.392,0.629,5,-7.056,1,0.0335,0.718,0.000004,0.1290,0.720,124.511,audio_features,2Gy7qnDwt8Z3MNxqat4CsK,spotify:track:2Gy7qnDwt8Z3MNxqat4CsK,https://api.spotify.com/v1/tracks/2Gy7qnDwt8Z3...,https://api.spotify.com/v1/audio-analysis/2Gy7...,153205,4


(6246, 20)

### Adding a third playlist

In [76]:
all_tracks_3 = get_playlist_tracks("77CxoauZX3xv8VsfVwPJxT")

In [77]:
len(all_tracks_3)

1716

In [78]:
spotify_df_3=playlist_to_df(all_tracks_3)

In [79]:
display(spotify_df_3.head())
display(spotify_df_3.shape)


,artist,song,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Lost Frequencies,Like I Love You,0.863,0.588,1,-7.141,1,0.0847,0.1420,0.000034,0.4570,0.751,117.911,audio_features,4sh506lJZac1T4o4NqtIuS,spotify:track:4sh506lJZac1T4o4NqtIuS,https://api.spotify.com/v1/tracks/4sh506lJZac1...,https://api.spotify.com/v1/audio-analysis/4sh5...,190174,4
1,Shawn Mendes,Lost In Japan - Remix,0.706,0.855,10,-5.377,1,0.2150,0.0911,0.000000,0.3470,0.293,107.184,audio_features,575NJxNUVDqwJGdzBrlLbv,spotify:track:575NJxNUVDqwJGdzBrlLbv,https://api.spotify.com/v1/tracks/575NJxNUVDqw...,https://api.spotify.com/v1/audio-analysis/575N...,201253,4
2,Dua Lipa,IDGAF,0.836,0.544,7,-5.975,1,0.0943,0.0403,0.000000,0.0824,0.510,97.028,audio_features,76cy1WJvNGJTj78UqeA5zr,spotify:track:76cy1WJvNGJTj78UqeA5zr,https://api.spotify.com/v1/tracks/76cy1WJvNGJT...,https://api.spotify.com/v1/audio-analysis/76cy...,217947,4
3,Rita Ora,Your Song,0.855,0.624,1,-4.093,1,0.0488,0.1580,0.000000,0.0513,0.962,117.959,audio_features,4c2W3VKsOFoIg2SFaO6DY5,spotify:track:4c2W3VKsOFoIg2SFaO6DY5,https://api.spotify.com/v1/tracks/4c2W3VKsOFoI...,https://api.spotify.com/v1/audio-analysis/4c2W...,180757,4
4,Julia Michaels,Issues,0.670,0.415,8,-6.855,1,0.0786,0.3900,0.000000,0.0617,0.445,114.212,audio_features,0xG0SHshCStMEyNbNC5d4k,spotify:track:0xG0SHshCStMEyNbNC5d4k,https://api.spotify.com/v1/tracks/0xG0SHshCStM...,https://api.spotify.com/v1/audio-analysis/0xG0...,176320,4


(1716, 20)

### Adding a fourth playlist

In [80]:
all_tracks_4 = get_playlist_tracks("6sNDAFPubg3k4CuyH1fqrR")

In [81]:
len(all_tracks_4)

1015

In [82]:
spotify_df_4=playlist_to_df(all_tracks_4)

In [83]:
display(spotify_df_4.head())
display(spotify_df_4.shape)


,artist,song,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Various Artists,"Becoming one of ""The People"" Becoming one with...",0.0993,0.0902,1,-26.307,1,0.0428,0.692,0.8500,0.0745,0.0382,71.062,audio_features,1UBd7aWHZCz8yirI9KCvAU,spotify:track:1UBd7aWHZCz8yirI9KCvAU,https://api.spotify.com/v1/tracks/1UBd7aWHZCz8...,https://api.spotify.com/v1/audio-analysis/1UBd...,463707,3
1,James Horner,For The Love Of A Princess,0.1870,0.0602,9,-25.767,0,0.0464,0.408,0.9220,0.1220,0.0362,68.751,audio_features,22yxHt6UqZpH7tP6W4PooI,spotify:track:22yxHt6UqZpH7tP6W4PooI,https://api.spotify.com/v1/tracks/22yxHt6UqZpH...,https://api.spotify.com/v1/audio-analysis/22yx...,247333,4
2,Various Artists,"The Council of Elrond (feat. ""Aniron"") [Theme ...",0.1040,0.1110,1,-20.031,1,0.0517,0.979,0.9260,0.1100,0.0390,83.933,audio_features,3Knohqfb9jeYzL6wMZiWLM,spotify:track:3Knohqfb9jeYzL6wMZiWLM,https://api.spotify.com/v1/tracks/3Knohqfb9jeY...,https://api.spotify.com/v1/audio-analysis/3Kno...,229107,4
3,The Lyndhurst Orchestra,Now We Are Free,0.3280,0.2190,9,-18.595,1,0.0337,0.579,0.0991,0.1040,0.0387,138.022,audio_features,11IXRdEjiH0KL8nh4G1j7d,spotify:track:11IXRdEjiH0KL8nh4G1j7d,https://api.spotify.com/v1/tracks/11IXRdEjiH0K...,https://api.spotify.com/v1/audio-analysis/11IX...,254293,4
4,Alan Silvestri,Suite from Forrest Gump,0.1110,0.1030,5,-18.360,1,0.0356,0.893,0.8160,0.1640,0.0448,84.520,audio_features,435vx38BVoUIqSG1VfoJOh,spotify:track:435vx38BVoUIqSG1VfoJOh,https://api.spotify.com/v1/tracks/435vx38BVoUI...,https://api.spotify.com/v1/audio-analysis/435v...,394240,4


(1015, 20)

### Adding a fifth playlist

In [84]:
all_tracks_5 = get_playlist_tracks("7B0kSnc5JgV8ljvBAgsKD8")

In [85]:
len(all_tracks_5)

2742

In [86]:
spotify_df_5=playlist_to_df(all_tracks_5)

In [87]:
display(spotify_df_5.head())
display(spotify_df_5.shape)

,artist,song,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Khalid,Let's Go,0.780,0.371,9,-6.705,1,0.0559,0.5720,0.000000,0.247,0.430,108.003,audio_features,6AIccJIWHGK26xi34FAkwl,spotify:track:6AIccJIWHGK26xi34FAkwl,https://api.spotify.com/v1/tracks/6AIccJIWHGK2...,https://api.spotify.com/v1/audio-analysis/6AIc...,204547,4
1,Chance the Rapper,Smoke Break (feat. Future),0.705,0.453,3,-8.174,1,0.1960,0.2570,0.000000,0.143,0.283,124.942,audio_features,1Uq3IOIy1CUlHUgP6vWpum,spotify:track:1Uq3IOIy1CUlHUgP6vWpum,https://api.spotify.com/v1/tracks/1Uq3IOIy1CUl...,https://api.spotify.com/v1/audio-analysis/1Uq3...,226494,4
2,Ariana Grande,Side To Side,0.650,0.736,6,-5.840,0,0.2290,0.0528,0.000000,0.235,0.613,159.173,audio_features,1pKeFVVUOPjFsOABub0OaV,spotify:track:1pKeFVVUOPjFsOABub0OaV,https://api.spotify.com/v1/tracks/1pKeFVVUOPjF...,https://api.spotify.com/v1/audio-analysis/1pKe...,226160,4
3,The Weeknd,Die For You,0.582,0.525,1,-7.163,0,0.0627,0.1110,0.000000,0.134,0.510,133.249,audio_features,2Ch7LmS7r2Gy2kc64wv3Bz,spotify:track:2Ch7LmS7r2Gy2kc64wv3Bz,https://api.spotify.com/v1/tracks/2Ch7LmS7r2Gy...,https://api.spotify.com/v1/audio-analysis/2Ch7...,260253,4
4,Madison Beer,Something Sweet,0.622,0.791,9,-5.010,0,0.0377,0.1820,0.000003,0.245,0.647,99.979,audio_features,3PlM3biKS6ZUIHojCJmElE,spotify:track:3PlM3biKS6ZUIHojCJmElE,https://api.spotify.com/v1/tracks/3PlM3biKS6ZU...,https://api.spotify.com/v1/audio-analysis/3PlM...,195560,4


(2742, 20)

### Adding a sixth playlist

In [88]:
all_tracks_6 = get_playlist_tracks("1DG8jE9e1QeJU9xfuU7KEv")

In [89]:
len(all_tracks_6)

1582

In [90]:
spotify_df_6=playlist_to_df(all_tracks_6)

### Adding a seventh playlist

In [91]:
all_tracks_7 = get_playlist_tracks("3ANGJ9boEGZkOzCx5D1LRA")

In [92]:
len(all_tracks_7)

290

In [93]:
spotify_df_7=playlist_to_df(all_tracks_7)

### Adding a eight playlist

In [94]:
all_tracks_8 = get_playlist_tracks("7v2bpIgH3Yk6CQbpp3YXKu")

In [95]:
len(all_tracks_8)

2121

In [96]:
spotify_df_8=playlist_to_df(all_tracks_8)

### Adding a nineth playlist

In [97]:
all_tracks_9 = get_playlist_tracks("37i9dQZF1E8Lk8LqShtYPE")

In [98]:
len(all_tracks_9)

50

In [99]:
spotify_df_9=playlist_to_df(all_tracks_9)

### Adding a tenth playlist

In [100]:
all_tracks_10 = get_playlist_tracks("2wb9pCHSxGuVLYHsUBAjnR")

In [101]:
len(all_tracks_10)

318

In [102]:
spotify_df_10=playlist_to_df(all_tracks_10)

### Adding a eleventh playlist

In [103]:
all_tracks_11 = get_playlist_tracks("5oKz4DsTP8zbL97UIPbqp4")

In [104]:
len(all_tracks_11)

434

In [105]:
spotify_df_11=playlist_to_df(all_tracks_11)

### Adding a twelfth playlist

In [106]:
all_tracks_12 = get_playlist_tracks("37i9dQZF1DWVAa4Dwc5f7L")

In [107]:
len(all_tracks_12)

100

In [108]:
spotify_df_12=playlist_to_df(all_tracks_12)

### Adding a 13th playlist

In [109]:
all_tracks_13 = get_playlist_tracks("2PjVPkj4a9kBvQIXaZ6UUt")

In [110]:
len(all_tracks_13)

84

In [111]:
spotify_df_13=playlist_to_df(all_tracks_13)

### Adding a 14th playlist

In [112]:
all_tracks_14 = get_playlist_tracks("1CAT5I8rHPpvK33UONeKAP")

In [113]:
len(all_tracks_14)

83

In [114]:
spotify_df_14=playlist_to_df(all_tracks_14)

### Adding a 15th playlist

In [115]:
all_tracks_15 = get_playlist_tracks("37i9dQZF1DWZLN2cXno63R")

In [116]:
len(all_tracks_15)

62

In [117]:
spotify_df_15=playlist_to_df(all_tracks_15)

### Adding a 16th playlist

In [118]:
all_tracks_16 = get_playlist_tracks("37i9dQZF1DZ06evO343CE2")

In [119]:
len(all_tracks_16)

50

In [120]:
spotify_df_16=playlist_to_df(all_tracks_16)

### Adding a 17th playlist

In [121]:
all_tracks_17 = get_playlist_tracks("37i9dQZF1DWWPg9Mo6aXVl")

In [122]:
len(all_tracks_17)

100

In [123]:
spotify_df_17=playlist_to_df(all_tracks_17)

### Adding a 18th playlist

In [124]:
all_tracks_18 = get_playlist_tracks("5GzKZGlIbWhyfs9vUvrpIU")

In [125]:
len(all_tracks_18)

66

In [126]:
spotify_df_18=playlist_to_df(all_tracks_18)

### Adding a 19th playlist

In [127]:
all_tracks_19 = get_playlist_tracks("6ttaSnMvNlLeNf2hC8p6f2")

In [128]:
len(all_tracks_19)

212

In [129]:
spotify_df_19=playlist_to_df(all_tracks_19)

### Adding a 20th playlist

In [130]:
all_tracks_20 = get_playlist_tracks("4gohFBGkxw3Cr8S4e3gPJr")

In [131]:
len(all_tracks_20)

174

In [132]:
spotify_df_20=playlist_to_df(all_tracks_20)

### Adding a 21st playlist

In [133]:
all_tracks_21 = get_playlist_tracks("5wc7UIul4pdHmP8d1KMjVJ")

In [134]:
len(all_tracks_21)

224

In [135]:
spotify_df_21=playlist_to_df(all_tracks_21)

### Adding a 22nd playlist

In [136]:
all_tracks_22 = get_playlist_tracks("1h0CEZCm6IbFTbxThn6Xcs")

In [137]:
len(all_tracks_22)

166

In [138]:
spotify_df_22=playlist_to_df(all_tracks_22)

### Adding a 23rd playlist

In [139]:
all_tracks_23 = get_playlist_tracks("2A59Os98FB0oVDb2xY3N9T")

In [140]:
len(all_tracks_23)

309

In [141]:
spotify_df_23=playlist_to_df(all_tracks_23)

### Concatenating the dataframes

In [142]:
spotify_df = pd.concat([spotify_df_1, spotify_df_2, spotify_df_3, spotify_df_4, spotify_df_5,
                       spotify_df_6, spotify_df_7, spotify_df_8, spotify_df_9,spotify_df_10,
                       spotify_df_11, spotify_df_12, spotify_df_13,spotify_df_14, spotify_df_15,
                       spotify_df_16, spotify_df_17, spotify_df_18, spotify_df_19, spotify_df_20,
                       spotify_df_21, spotify_df_22, spotify_df_23], axis=0)

In [143]:
display(spotify_df.head())
display(spotify_df.shape)

,artist,song,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,John Lennon,Imagine - Remastered 2010,0.547,0.257,0,-12.358,1,0.0252,0.907,0.1830,0.0935,0.169,75.752,audio_features,7pKfPomDEeI4TPT6EOYjn9,spotify:track:7pKfPomDEeI4TPT6EOYjn9,https://api.spotify.com/v1/tracks/7pKfPomDEeI4...,https://api.spotify.com/v1/audio-analysis/7pKf...,187867,4
1,Procol Harum,A Whiter Shade Of Pale,0.249,0.660,0,-6.905,1,0.0342,0.504,0.0026,0.0891,0.435,149.813,audio_features,3qitymULqEibr7yknRMKU4,spotify:track:3qitymULqEibr7yknRMKU4,https://api.spotify.com/v1/tracks/3qitymULqEib...,https://api.spotify.com/v1/audio-analysis/3qit...,248947,4
2,Queen,Bohemian Rhapsody - Remastered 2011,0.414,0.404,0,-9.928,0,0.0499,0.271,0.0000,0.3000,0.224,71.105,audio_features,1AhDOtG9vPSOmsWgNW0BEY,spotify:track:1AhDOtG9vPSOmsWgNW0BEY,https://api.spotify.com/v1/tracks/1AhDOtG9vPSO...,https://api.spotify.com/v1/audio-analysis/1AhD...,354320,4
3,Pink Floyd,Comfortably Numb - 2011 Remastered Version,0.472,0.366,11,-12.595,0,0.0286,0.150,0.3080,0.0837,0.171,127.167,audio_features,082cLCIXNPg2ruTrENz4Vt,spotify:track:082cLCIXNPg2ruTrENz4Vt,https://api.spotify.com/v1/tracks/082cLCIXNPg2...,https://api.spotify.com/v1/audio-analysis/082c...,382297,4
4,Bob Dylan,Knockin' On Heaven's Door,0.513,0.396,7,-13.061,1,0.0299,0.251,0.1770,0.1100,0.229,140.208,audio_features,6HSXNV0b4M4cLJ7ljgVVeh,spotify:track:6HSXNV0b4M4cLJ7ljgVVeh,https://api.spotify.com/v1/tracks/6HSXNV0b4M4c...,https://api.spotify.com/v1/audio-analysis/6HSX...,149880,4


(18646, 20)

In [144]:
spotify_df.drop_duplicates(subset=['artist','song'],inplace=True)

display(spotify_df.shape) #Unique songs!!!

(16198, 20)

In [145]:
spotify_df.to_csv('spotify_df.csv')